# 05 - Latches and Flip-Flops: Memory Elements

## Introduction

Until now, everything we've built has been **combinational** - the output depends only on the current inputs. But computers need **memory** - the ability to store and remember values.

In this notebook, we'll build the fundamental memory elements:

- **SR Latch** - The simplest memory element
- **Gated SR Latch** - SR Latch with enable control
- **D Latch** - Data latch (level-triggered)
- **D Flip-Flop** - Edge-triggered memory (the building block of registers)
- **JK Flip-Flop** - Versatile flip-flop with toggle capability
- **T Flip-Flop** - Toggle flip-flop (for counters)

## Learning Objectives

1. Understand the difference between combinational and sequential logic
2. Learn how feedback creates memory
3. Understand level-triggered vs edge-triggered behavior
4. Build flip-flops from basic gates

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))
sys.path.insert(0, str(project_root))

print("Setup complete!")

## Sequential vs Combinational Logic

| Combinational | Sequential |
|--------------|------------|
| Output depends only on current inputs | Output depends on inputs AND previous state |
| No memory | Has memory (state) |
| AND, OR, NOT, MUX, ALU | Latches, Flip-Flops, Registers |

**Key insight**: Memory is created by **feedback** - connecting an output back to an input.

## The SR Latch

The **SR (Set-Reset) Latch** is the simplest memory element. It's built from two cross-coupled NOR gates:

```mermaid
flowchart LR
    R((R)) --> NOR1[NOR]
    NOR1 --> Q((Q))
    Q --> NOR2[NOR]
    S((S)) --> NOR2
    NOR2 --> Qbar(("Q̄"))
    Qbar --> NOR1
```

### Truth Table

| S | R | Q (next) | Action |
|---|---|----------|--------|
| 0 | 0 | Q (prev) | Hold (no change) |
| 1 | 0 | 1 | Set |
| 0 | 1 | 0 | Reset |
| 1 | 1 | ? | Invalid (avoid!) |

## Exercise 1: SR Latch

Implement an SR Latch using cross-coupled NOR gates.

**Hint**: The output feeds back as input to the other gate. You may need to iterate a few times for the values to stabilize.

In [ ]:
class SRLatch:
    """SR (Set-Reset) Latch using NOR gates."""
    
    def __init__(self):
        self.q = 0
        self.q_bar = 1
    
    def __call__(self, s: int, r: int) -> int:
        """
        Update latch state and return Q.
        
        Args:
            s: Set input (1 to set Q to 1)
            r: Reset input (1 to reset Q to 0)
        
        Returns:
            Current Q value
        """
        # YOUR CODE HERE
        # Use NOR gates with feedback
        # Iterate to stabilize: Q = NOR(R, Q_bar), Q_bar = NOR(S, Q)
        pass

# Test
sr = SRLatch()
print(f"Initial Q: {sr.q}")
print(f"Set (S=1, R=0): Q = {sr(1, 0)}")
print(f"Hold (S=0, R=0): Q = {sr(0, 0)}")
print(f"Reset (S=0, R=1): Q = {sr(0, 1)}")
print(f"Hold (S=0, R=0): Q = {sr(0, 0)}")

## The Gated SR Latch

The SR Latch responds immediately to inputs. Sometimes we want to control **when** the latch can change. The Gated SR Latch adds an **enable** signal:

```mermaid
flowchart LR
    S((S)) --> AND1[AND]
    E((E)) --> AND1
    E --> AND2[AND]
    R((R)) --> AND2
    AND1 --> SR[SR Latch]
    AND2 --> SR
    SR --> Q((Q))
```

When E=0, the latch ignores S and R. When E=1, it behaves like a normal SR latch.

## Exercise 2: Gated SR Latch

In [ ]:
class GatedSRLatch:
    """Gated SR Latch."""
    
    def __init__(self):
        self.sr_latch = SRLatch()
    
    def __call__(self, s: int, r: int, enable: int) -> int:
        """
        Update latch when enabled.
        
        Args:
            s: Set input
            r: Reset input
            enable: When 1, latch responds to S and R
        """
        # YOUR CODE HERE
        # Gate S and R with enable, then pass to SR latch
        pass

# Test
gated = GatedSRLatch()
print(f"Enable=0, Set: Q = {gated(1, 0, 0)}")
print(f"Enable=1, Set: Q = {gated(1, 0, 1)}")
print(f"Enable=0, Reset: Q = {gated(0, 1, 0)}")
print(f"Enable=1, Reset: Q = {gated(0, 1, 1)}")

## The D Latch

The SR Latch has an invalid state (S=1, R=1). The **D (Data) Latch** solves this by using a single data input:

```mermaid
flowchart LR
    D((D)) --> AND1[AND]
    E((E)) --> AND1
    D --> NOT[NOT]
    NOT --> AND2[AND]
    E --> AND2
    AND1 --> SR[SR Latch]
    AND2 --> SR
    SR --> Q((Q))
```

- When E=1: Q follows D ("transparent")
- When E=0: Q holds its value ("latched")

## Exercise 3: D Latch

In [ ]:
class DLatch:
    """D (Data) Latch - level-triggered."""
    
    def __init__(self):
        self.q = 0
    
    def __call__(self, d: int, enable: int) -> int:
        """
        When enabled, Q follows D. When disabled, Q holds.
        
        Args:
            d: Data input
            enable: When 1, Q follows D
        """
        # YOUR CODE HERE
        pass

# Test
dlatch = DLatch()
print(f"E=1, D=1: Q = {dlatch(1, 1)}")
print(f"E=0, D=0: Q = {dlatch(0, 0)} (should hold 1)")
print(f"E=1, D=0: Q = {dlatch(0, 1)}")

## Level-Triggered vs Edge-Triggered

The D Latch is **level-triggered** - it's transparent whenever enable is high. This can cause problems in synchronous circuits where we want data to change only at specific moments.

The **D Flip-Flop** is **edge-triggered** - it captures data only on the rising edge of the clock (when clock goes from 0 to 1).

```mermaid
flowchart LR
    subgraph Level["Level-Triggered (Latch)"]
        L1[Transparent entire time CLK is high]
    end
    subgraph Edge["Edge-Triggered (Flip-Flop)"]
        E1[Captures only at rising edge ↑]
    end
```

## Exercise 4: D Flip-Flop

In [ ]:
class DFlipFlop:
    """D Flip-Flop - edge-triggered on rising edge."""
    
    def __init__(self):
        self.q = 0
        self._prev_clk = 0
    
    def clock(self, d: int, clk: int) -> int:
        """
        Capture D on rising edge of clock.
        
        Args:
            d: Data input
            clk: Clock signal
        
        Returns:
            Current Q value
        """
        # YOUR CODE HERE
        # Detect rising edge: prev_clk=0 AND clk=1
        # On rising edge, capture D
        pass
    
    def read(self) -> int:
        return self.q

# Test
dff = DFlipFlop()
print("D=1, CLK: 0->1 (rising edge)")
dff.clock(1, 0)  # CLK low
print(f"  After CLK=0: Q = {dff.read()}")
dff.clock(1, 1)  # CLK high (rising edge!)
print(f"  After CLK=1: Q = {dff.read()}")
print("\nD=0, CLK: 1->0->1")
dff.clock(0, 0)  # CLK low
print(f"  After CLK=0: Q = {dff.read()} (should hold 1)")
dff.clock(0, 1)  # Rising edge with D=0
print(f"  After CLK=1: Q = {dff.read()} (captures 0)")

## The JK Flip-Flop

The **JK Flip-Flop** is more versatile than the D Flip-Flop. It has two inputs (J and K):

| J | K | Q (next) | Action |
|---|---|----------|--------|
| 0 | 0 | Q | Hold |
| 0 | 1 | 0 | Reset |
| 1 | 0 | 1 | Set |
| 1 | 1 | Q̄ | Toggle |

Unlike the SR latch, J=K=1 is valid and useful - it toggles the output!

## Exercise 5: JK Flip-Flop

In [ ]:
class JKFlipFlop:
    """JK Flip-Flop - edge-triggered."""
    
    def __init__(self):
        self.q = 0
        self._prev_clk = 0
    
    def clock(self, j: int, k: int, clk: int) -> int:
        """
        JK operation on rising edge.
        
        J=0,K=0: Hold
        J=0,K=1: Reset (Q=0)
        J=1,K=0: Set (Q=1)
        J=1,K=1: Toggle
        """
        # YOUR CODE HERE
        pass
    
    def read(self) -> int:
        return self.q

# Test
jk = JKFlipFlop()
print(f"Initial: Q = {jk.read()}")
jk.clock(1, 0, 0); jk.clock(1, 0, 1)  # Set
print(f"J=1,K=0 (Set): Q = {jk.read()}")
jk.clock(1, 1, 0); jk.clock(1, 1, 1)  # Toggle
print(f"J=1,K=1 (Toggle): Q = {jk.read()}")
jk.clock(1, 1, 0); jk.clock(1, 1, 1)  # Toggle again
print(f"J=1,K=1 (Toggle): Q = {jk.read()}")
jk.clock(0, 1, 0); jk.clock(0, 1, 1)  # Reset
print(f"J=0,K=1 (Reset): Q = {jk.read()}")

## The T Flip-Flop

The **T (Toggle) Flip-Flop** is a simplified JK flip-flop where J=K=T:

| T | Q (next) | Action |
|---|----------|--------|
| 0 | Q | Hold |
| 1 | Q̄ | Toggle |

T flip-flops are perfect for building **counters** - each flip-flop divides the input frequency by 2.

## Exercise 6: T Flip-Flop

In [ ]:
class TFlipFlop:
    """T (Toggle) Flip-Flop - built from JK flip-flop."""
    
    def __init__(self):
        self.jk = JKFlipFlop()
    
    def clock(self, t: int, clk: int) -> int:
        """
        T=0: Hold
        T=1: Toggle
        """
        # YOUR CODE HERE
        # Hint: Use the JK flip-flop with J=K=T
        pass
    
    def read(self) -> int:
        return self.jk.read()

# Test - count in binary!
t = TFlipFlop()
print("Toggling T flip-flop:")
for i in range(8):
    t.clock(1, 0)
    q = t.clock(1, 1)
    print(f"  Toggle {i+1}: Q = {q}")

## Copy Your Implementation

Once your code works, copy all the classes to `src/computer/sequential.py`

## Validation

In [ ]:
from utils.checker import check
check('sequential')

## Summary

We've built the fundamental memory elements:

| Component | Trigger | Use Case |
|-----------|---------|----------|
| SR Latch | Level | Simple memory |
| D Latch | Level | Temporary storage |
| D Flip-Flop | Edge | Registers |
| JK Flip-Flop | Edge | Versatile operations |
| T Flip-Flop | Edge | Counters |

### Key Concepts

1. **Feedback** creates memory
2. **Level-triggered** = transparent when enabled
3. **Edge-triggered** = captures only at clock transition
4. **D Flip-Flop** is the building block of registers

### What's Next?

In the next notebook, we'll combine multiple D flip-flops to build **registers** - the CPU's fast local storage!